### Getters and Setters

До сих пор мы видели, как вызывается метод `__get__`, когда мы назначаем экземпляр дескриптора атрибуту класса.

Но мы можем получить доступ к этому атрибуту либо из самого класса, либо из экземпляра — как мы видели, оба доступа заканчиваются вызовом метода `__get__`.

Но что меняют аргументы, переданные методу. Давайте рассмотрим это:

In [1]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        print(f'__get__ called, self={self}, instance={instance}, owner_class={owner_class}')
        return datetime.utcnow().isoformat()

In [2]:
class Logger1:
    current_time = TimeUTC()

class Logger2:
    current_time = TimeUTC()

Теперь давайте получим доступ к `current_time` из самого класса:

In [3]:
Logger1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x7f83d035be48>, instance=None, owner_class=<class '__main__.Logger1'>


'2019-07-13T20:47:14.961760'

Как вы можете видеть, `instance` был `None` - это произошло потому, что мы вызвали дескриптор из класса `Logger1`, а не его экземпляр. `owner_class` сообщает нам, что этот экземпляр дескриптора определен в классе `Logger1`.

То же самое происходит, если мы используем `Logger2`:

In [4]:
Logger2.current_time

__get__ called, self=<__main__.TimeUTC object at 0x7f83d035be80>, instance=None, owner_class=<class '__main__.Logger2'>


'2019-07-13T20:47:14.997577'

Но если вместо этого мы вызовем дескриптор через экземпляр:

In [5]:
l1 = Logger1()
print(hex(id(l1)))

0x7f83d03864a8


In [6]:
l1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x7f83d035be48>, instance=<__main__.Logger1 object at 0x7f83d03864a8>, owner_class=<class '__main__.Logger1'>


'2019-07-13T20:47:15.027484'

Как вы можете видеть, `instance` теперь является экземпляром `l1`, а классом-владельцем по-прежнему является `Logger1`.

SME выполняется для экземпляра `Logger2`:

In [7]:
l2 = Logger2()
print(hex(id(l2)))
l2.current_time

0x7f83d0386b38
__get__ called, self=<__main__.TimeUTC object at 0x7f83d035be80>, instance=<__main__.Logger2 object at 0x7f83d0386b38>, owner_class=<class '__main__.Logger2'>


'2019-07-13T20:47:15.043101'

Это означает, что мы можем различать внутри нашего метода `__get__`, был ли доступ к дескриптору осуществлен через класс или через экземпляр.

Обычно, когда к дескриптору осуществляется доступ из класса, мы возвращаем экземпляр дескриптора, а при доступе из экземпляра мы возвращаем нужное нам значение, специфичное для экземпляра:

In [8]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            # called from class
            return self
        else:
            # called from instance
            return datetime.utcnow().isoformat()

In [9]:
class Logger:
    current_time = TimeUTC()

In [10]:
Logger.current_time

In [11]:
l = Logger()

In [12]:
l.current_time

'2019-07-13T20:47:15.109595'

Это соответствует принципу работы свойств:

In [13]:
class Logger:
    @property
    def current_time(self):
        return datetime.utcnow().isoformat()

In [14]:
Logger.current_time

Это вернуло экземпляр свойства, тогда как вызов его из экземпляра:

In [15]:
l = Logger()
l.current_time

'2019-07-13T20:47:15.162299'

Теперь есть один тонкий момент, который нам следует понимать при создании нескольких экземпляров класса, использующего дескриптор в качестве атрибута класса.

Поскольку дескриптор назначен **атрибуту класса**, все экземпляры класса будут **использовать** один и тот же экземпляр дескриптора!

In [16]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            # called from class
            return self
        else:
            # called from instance
            print(f'__get__ called in {self}')
            return datetime.utcnow().isoformat()

class Logger:
    current_time = TimeUTC()

In [17]:
l1 = Logger()
l2 = Logger()

Но посмотрите на `current_time` для каждого из этих случаев.

In [18]:
l1.current_time, l2.current_time

__get__ called in <__main__.TimeUTC object at 0x7f83d039aeb8>
__get__ called in <__main__.TimeUTC object at 0x7f83d039aeb8>


('2019-07-13T20:47:15.209930', '2019-07-13T20:47:15.210094')

Но посмотрите на `current_time` для каждого из этих экземпляров. Как вы можете видеть, использовался **один и тот же** экземпляр `TimeUTC`.

В данном конкретном примере это не имеет значения, поскольку мы просто возвращаем текущее время, но посмотрите, что произойдет, если наше свойство будет зависеть от какого-либо состояния в дескрипторе:

In [19]:
class Countdown:
    def __init__(self, start):
        self.start = start + 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            self.start -= 1
            return self.start

In [20]:
class Rocket:
    countdown = Countdown(10)

Теперь предположим, что мы хотим запустить две ракеты:

In [21]:
rocket1 = Rocket()
rocket2 = Rocket()

И давайте начнем обратный отсчет для каждого из них:

In [22]:
rocket1.countdown

10

In [23]:
rocket2.countdown

9

In [24]:
rocket1.countdown

8

Как вы можете видеть, текущее значение обратного отсчета является общим для экземпляров `rocket1` и `rocket2` объекта `Rocket` - это потому, что экземпляр `Countdown` является атрибутом класса `Rocket`. Поэтому мы должны быть осторожны, когда работаем с состоянием уровня экземпляра.

Метод `__set__` работает аналогично `__get__`, но он используется, когда мы присваиваем значение атрибуту класса.

In [25]:
class IntegerValue:
    def __set__(self, instance, value):
        print(f'__set__ called, instance={instance}, value={value}')

    def __get__(self, instance, owner_class):
        if instance is None:
            print('__get__ called from class')
        else:
            print(f'__get__ called, instance={instance}, owner_class={owner_class}')

In [26]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [27]:
Point2D.x

__get__ called from class


In [28]:
p = Point2D()

In [29]:
p.x

__get__ called, instance=<__main__.Point2D object at 0x7f83d03a8f28>, owner_class=<class '__main__.Point2D'>


In [30]:
p.x = 100

__set__ called, instance=<__main__.Point2D object at 0x7f83d03a8f28>, value=100


Итак, где нам следует хранить значения `x` и `y`?

Многие «руководства», которые я вижу в Интернете, наивно хранят значение в самом дескрипторе:

In [31]:
class IntegerValue:
    def __set__(self, instance, value):
        self._value = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self._value

In [32]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

На первый взгляд, это работает просто отлично:

In [33]:
p1 = Point2D()

In [34]:
p1.x = 1.1
p1.y = 2.2

In [35]:
p1.x, p1.y

(1, 2)

Но помните, что я говорил о том, что экземпляр дескриптора (в данном случае `IntegeraValue`) является общим для всех экземпляров класса (в данном случае `Point2D`)?

In [36]:
p2 = Point2D()

In [37]:
p2.x, p2.y

(1, 2)

И, конечно, если мы установим значение:

In [38]:
p2.x = 100.9

In [39]:
p2.x, p1.x

(100, 100)

Таким образом, очевидно, что использование словаря экземпляра дескриптора для хранения на уровне экземпляра, вероятно, не будет работать в большинстве случаев!

Вот почему методы `__get__` и `__set__` должны знать, с каким экземпляром мы имеем дело.

---